# Spatial Join for Land Use and Sensor Readings

## Spatial data pulled from MassGIS OLIVER tool
#### Uses 2005 land use revision data

In [2]:
import pandas as pd
import numpy as np
import geopandas as gp
import sys
import matplotlib.pyplot as plt
%pylab inline
print sys.version

Populating the interactive namespace from numpy and matplotlib
2.7.11 (default, Dec  5 2015, 14:44:47) 
[GCC 4.2.1 Compatible Apple LLVM 7.0.0 (clang-700.1.76)]


In [3]:
# read in census block data and 2005 land use data
# first, polygons
blocks = gp.read_file('land_data/Census 2010 Blocks/GISDATA_CENSUS2010BLOCKS_POLY.shp')
LU2005 = gp.read_file('land_data/Land Use 2005/GISDATA_LANDUSE2005_POLY.shp')
landuse=gp.read_file('landuse/GISDATA_LANDUSE_POLY.shp')
# then, points
df = pd.read_csv('nUrve_master.csv')

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (13,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# import necessary modules for the spatial join
import os
from shapely.geometry import Point
from geopandas import GeoDataFrame, read_file
from geopandas.tools import overlay
from geopandas.tools import sjoin

In [5]:
# spatial joins take a while with so many points
def shapeMerge(points, polygons):
    points['geometry'] = points.apply(lambda x: Point(x.GPS_LON, x.GPS_LAT), axis =1)
    points = GeoDataFrame(points,crs = {'init': 'epsg:4326'})
    polygons = GeoDataFrame(polygons,crs = {'init': 'epsg:4326'})
    #polygons.crs = points.crs
    s_join = sjoin(points, polygons, how='left', op='within')
    return s_join

In [6]:
# merge points with 2005 land use
joined = shapeMerge(df, LU2005)
joined.head()

,ID,GPS_DATETIMESTAMP,GPS_LAT,GPS_LON,GPS_Speed,GPS_Alt,GPS_Sats,GPS_Fix,GPS_Quality,AMB_Temp,...,RDQ_AcYMea,RDQ_AcZ,RDQ_AcZMin,RDQ_AcZMax,RDQ_AcZMea,SamplingCount,geometry,index_right,LU05_DESC,LUCODE
0,132,2015-10-26 2:48:45.0,42.34143,-71.080329,0.47,183.7,4,1,1,21.23,...,0.5672,9.1006,7.9238,9.2183,8.6780,255,POINT (-71.08032900000001 42.34143),4393,Commercial,15
1,134,2015-10-26 2:48:45.0,42.34143,-71.080329,0.47,183.7,4,1,1,21.23,...,0.5544,9.1790,8.2376,9.3359,8.7051,254,POINT (-71.08032900000001 42.34143),4393,Commercial,15
2,136,2015-10-26 2:48:45.0,42.34143,-71.080329,0.47,183.7,4,1,1,21.24,...,0.5233,8.3945,8.1199,9.1790,8.6665,255,POINT (-71.08032900000001 42.34143),4393,Commercial,15
3,138,2015-10-26 2:48:45.0,42.34143,-71.080329,0.47,183.7,4,1,1,21.23,...,0.5401,8.5514,8.1591,9.1006,8.6938,254,POINT (-71.08032900000001 42.34143),4393,Commercial,15
4,140,2015-10-26 2:48:45.0,42.34143,-71.080329,0.47,183.7,4,1,1,21.23,...,0.5836,8.6299,8.0022,9.5321,8.6882,255,POINT (-71.08032900000001 42.34143),4393,Commercial,15


In [7]:
joined.columns

Index([               u'ID', u'GPS_DATETIMESTAMP',           u'GPS_LAT',
                 u'GPS_LON',         u'GPS_Speed',           u'GPS_Alt',
                u'GPS_Sats',           u'GPS_Fix',       u'GPS_Quality',
                u'AMB_Temp',          u'AMB_Humd',           u'AMB_Lux',
                 u'AMB_Snd',        u'AMB_SndMin',        u'AMB_SndMax',
              u'AMB_SndMea',           u'RDQ_AcX',        u'RDQ_AcXMin',
              u'RDQ_AcXMax',        u'RDQ_AcXMea',           u'RDQ_AcY',
              u'RDQ_AcYMin',        u'RDQ_AcYMax',        u'RDQ_AcYMea',
                 u'RDQ_AcZ',        u'RDQ_AcZMin',        u'RDQ_AcZMax',
              u'RDQ_AcZMea',     u'SamplingCount',          u'geometry',
             u'index_right',         u'LU05_DESC',            u'LUCODE'],
      dtype='object')

In [8]:
print 'There are %s different land use types in the 2005 revision.' % len(np.unique(joined.LU05_DESC))
print ''
print 'Those types are: \n', np.unique(joined.LU05_DESC)

There are 19 different land use types in the 2005 revision.

Those types are: 
[nan u'Cemetery' u'Commercial' u'Cropland' u'Forest'
 u'High Density Residential' u'Industrial' u'Medium Density Residential'
 u'Multi-Family Residential' u'Nursery' u'Open Land'
 u'Participation Recreation' u'Saltwater Sandy Beach'
 u'Spectator Recreation' u'Transitional' u'Transportation'
 u'Urban Public/Institutional' u'Very Low Density Residential' u'Water']


/usr/local/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [14]:
joined.ID.groupby(joined.LU05_DESC).agg('count').sort(ascending=False, inplace=False)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting
  if __name__ == '__main__':


LU05_DESC
Commercial                      354340
Multi-Family Residential        124374
Transportation                   91063
Urban Public/Institutional       53506
Industrial                       24550
High Density Residential         10377
Participation Recreation          5613
Transitional                      1529
Open Land                          956
Medium Density Residential         730
Cemetery                           720
Nursery                            138
Saltwater Sandy Beach               92
Very Low Density Residential        65
Spectator Recreation                26
Cropland                            10
Water                                4
Forest                               2
dtype: int64

Based on the distribution of land use categories, we will focus on trying to classify the top five, excuding Transportation:

    Commercial
    Multi-Family Residential
    Urban Public/Institutional
    Industrial
    High Density Residential